In [5]:
import time
import os
from mspasspy.algorithms import signals
from mspasspy.ccore.algorithms.basic import TimeWindow
from mspasspy.algorithms.window import WindowData
from mspasspy.graphics import SeismicPlotter
from mspasspy.algorithms.RFdeconProcessor import RFdeconProcessor, RFdecon
from mspasspy.algorithms.window import scale
from mspasspy.db.client import DBClient
from mspasspy.db.database import Database

# initialize the dask scheduler and Database client
hostname = os.environ.get('HOSTNAME')
client = DBClient(hostname)
db=Database(client, 'usarraytest')
# define deconvolution processor
processor=RFdeconProcessor()

# define window function for seismogram
def window_seis(i):
    twin=TimeWindow(-20.0,150.0)
    t=i.t0 + 300
    i.ator(t)
    wd = None
    try:
        wd = WindowData(i,twin)
    except Exception as err:
        wd = "error"
    return wd

In [9]:
# input data size
record_num = 10
# start timing
t0=time.time()
cursor=db.wf_Seismogram.find({},limit=record_num)
# read -> detrend -> filter -> window -> deconvolution
for doc in cursor:
    seis = db.read_data(doc, collection='wf_Seismogram')
    seis['arrival.time'] = seis.t0 + 300
    seis = signals.detrend(seis,'demean')
    seis = signals.filter(seis,"bandpass",freqmin=0.01,freqmax=2.0)
    # there is a window data function in the RF decon
    # seis = window_seis(seis)
    seis = RFdecon(seis, processor)
t1=time.time()-t0
print('Elapsed time for processing all = ',t1)

Elapsed time for processing all =  57.28491187095642


In [10]:
import dask.bag
from dask.distributed import Client as DaskClient
from mspasspy.db.database import read_distributed_data

# initialize the dask client
daskclient = DaskClient(hostname + ':8786')
#print(daskclient.scheduler_info())

# input data size
record_num = 10
# start timing
t0=time.time()
cursor=db.wf_Seismogram.find({},limit=record_num)
# read -> detrend -> filter -> window -> deconvolution
data = read_distributed_data(db, cursor)
data_det = data.map(signals.detrend,'demean')
data_fil = data_det.map(signals.filter,"bandpass",freqmin=0.01,freqmax=2.0)
data_win = data_fil.map(window_seis)
data_decon = data_fil.map(RFdecon, processor)
data_list = data_decon.compute()
t1=time.time()-t0
print('Elapsed time for processing all = ',t1)

distributed.protocol.pickle - INFO - Failed to serialize (<function reify at 0x2ba38cd4c158>, (<function map_chunk at 0x2ba38cd4c510>, <function RFdecon at 0x2ba38e3208c8>, [(<function map_chunk at 0x2ba38cd4c510>, <function filter at 0x2ba3894f26a8>, [(<function map_chunk at 0x2ba38cd4c510>, <function detrend at 0x2ba3894f29d8>, [(<function map_chunk at 0x2ba38cd4c510>, <function read_distributed_data.<locals>.<lambda> at 0x2ba38f08d268>, ["('from_sequence-318001d8724ef6fa32b2cf40f12805e3', 2)"], None, {}), (<class 'itertools.repeat'>, 'demean')], None, {}), (<class 'itertools.repeat'>, 'bandpass')], None, {'freqmin': 0.01, 'freqmax': 2.0}), (<class 'itertools.repeat'>, <mspasspy.algorithms.RFdeconProcessor.RFdeconProcessor object at 0x2ba38e26ef98>)], None, {})). Exception: can't pickle mspasspy.ccore.algorithms.deconvolution.LeastSquareDecon objects
distributed.protocol.pickle - INFO - Failed to serialize (<function reify at 0x2ba38cd4c158>, (<function map_chunk at 0x2ba38cd4c510>, 

KilledWorker: ("('detrend-filter-lambda-RFdecon-6deddebd4dbe5d32c5bd8a701d29e16b', 2)", <Worker 'tcp://206.76.206.14:41902', name: tcp://206.76.206.14:41902, memory: 0, processing: 1>)